---

Created for [Pricing and Hedging Derivative Securities: Theory and Methods](https://book.derivative-securities.org/)

Authored by
- Kerry Back, Rice University
- Hong Liu, Washington University in St. Louis
- Mark Loewenstein, University of Maryland
 
---

<a target="_blank" href="https://colab.research.google.com/github/math-finance-book/book-code/blob/main/12_Binomial.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [ ]:

import plotly
from IPython.display import display, HTML

plotly.offline.init_notebook_mode(connected=True)
display(
    HTML(
        '<script type="text/javascript" async src="https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.1/MathJax.js?config=TeX-MML-AM_SVG"></script>'
    )
)

In [ ]:
import numpy as np
from scipy.special import comb
# Binomial Model for European Option

r = .1 # interest rate
sig = .2 # volatility
T = .5 # Expiration
q = div = 0.0 # Dividend
S0 = 42 # initial stock price
K = 40 # strike price
times =100 # Number of steps
dt = T/times
delt = np.exp(-div*dt)
a = np.exp(r*dt)*delt
u = np.exp(sig*np.sqrt(dt))
d = 1/u
pu  = (a-d)/(u-d)
pd = 1-pu
vec = np.arange(times + 1)
vec1 = np.array([1] * (times + 1))
S = np.array([0] * (times + 1))
S = S0*u**(2*vec-times*vec1)
C =  np.maximum(S-K*vec1,0*vec1)
CC =  comb((times)*vec1,(times)*vec1-vec)*pu**(vec)*pd**(times*vec1-vec)*C
Call = sum(CC)*np.exp(-r*T)
print('The Value of the European Call is=',Call)

In [ ]:
from scipy import stats
import numpy as np
from scipy.optimize import minimize, minimize_scalar

def blackscholes(S0, K, r, q, sig, T, call = True):
    '''Calculate option price using B-S formula.

    Args:
        S0 (num): initial price of underlying asset.
        K (num): strick price.
        sig (num): Black-Scholes volatility.
        T (num): maturity.
        call (bool): True returns call price, False returns put price.

    Returns:
        num
    '''
    d1 = (np.log(S0/K) + (r -q + sig**2/2) * T)/(sig*np.sqrt(T))
    d2 = d1 - sig*np.sqrt(T)
#     norm = sp.stats.norm
    norm = stats.norm
    if call:
        return np.exp(-q*T)*S0 * norm.cdf(d1,0,1) - K * np.exp(-r * T) * norm.cdf(d2,0, 1)
    else:
        return -np.exp(-q*T)*S0 * norm.cdf(-d1,0,1) +K * np.exp(-r * T) * norm.cdf(-d2,0, 1)

truebsc = blackscholes(S0,K,r, div, sig,T)
print('The exact Black Scholes Price is=', truebsc)

In [ ]:
for i in range(times+1):
    S[i] = S0*u ** (2*i-times)
    C[i] =  max(S[i]-K,0)

    CC[i] =  comb((times),(i))*pu ** (i)*pd ** (times-i)*C[i]



Call = sum(CC)*np.exp(-r*T)
print('The Value of the European Call is=',Call)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time
from math import pow, exp, sqrt
# parameters
# number of steps
n = 100
# interest rate
r = .1
# true drift
mu = .15
# volatility
sig = .2
# Initial Stock Price
S0 = 42
# Strike Price
K = 42
# Maturity
T = 0.5
# dividend yield
y = 0

# calculate parameters for binomial model
dt = T/n
delt = np.exp(-y*dt)
a = np.exp(r*dt) * delt
u = np.exp(sig*np.sqrt(dt))
d = 1/u
pu = (a-d)/(u-d)
pd = 1-pu
# Build vector of ending values
# and prices for which put is exercised
ex = np.zeros(n+1)
S = np.zeros(n+1)
AP = np.zeros(n+1)

for j in range(n+1):
    S[j] = S0*u**(2*j-n)
    AP[j] = max(K-S[j],0)
    if AP[j]>0:
        ex[n] = S[j]

In [ ]:
for i in range(n):
    S = np.zeros(n-i)
    P = np.zeros(n-i)
    PP = np.zeros(n-i)
    for j in range(n-i):
        S[j] = S0*u**(2*j-(n-i-1))
        #
        # P calculates the value of early exercise
        P[j] = max(K-S[j],0)
        #
        # PP calculates value of waiting using payoffs
        # from next period
        PP[j] = (pu*AP[j+1] + pd*AP[j])/a
        #
        # AP is the max of ealry exercise and waiting
        AP[j] = max(P[j],PP[j])
        #
        # ex is price where early exercise is optimal
        if P[j] > PP[j]:
            ex[n-i] = S[j]
        if ex[n-i]==0:
           ex[n-i]=np.nan
               

print('The value of the American Put is=',AP[0])
plt.figure(figsize=(9,6))
plt.plot(dt*np.arange(n+1),ex)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time
from math import pow, exp, sqrt
# parameters
# number of steps
n = 100
# interest rate
r = .1
# true drift
mu = .15
# volatility
sig = .2
# Initial Stock Price
S0 = 42
# Strike Price
K = 42
# Maturity
T = 0.5
# dividend yield
y = 0

dt = T/n
delt = np.exp(-y*dt)
a = np.exp(r*dt) * delt
u = np.exp(sig*np.sqrt(dt))
d = 1/u
pu = (a-d)/(u-d)
pd = 1-pu
# Build vector of ending values
# and prices for which put is exercised
vec = np.arange(n+1)
vec1 = np.ones(n+1)

S = S0 * u**(2*vec - n*vec1)

AP = np.maximum(K-S,0)
#print(AP)
ex = S[AP>0]
eb = np.zeros(n+1)
eb[n] = ex.max()
# Backward recursion in the loop
for i in range(n):
    vec = np.arange(n-i)
    vec1 = np.ones(n-i)
    # Possible Stock prices at times-i period
    S = S0 * u**(2*vec-(n-i)*vec1+1)
#     S = S0 * u**(2*vec-(n-i))
    # P calculates the value of early exercise
    P = np.maximum(K*vec1 - S, 0)
    # PP calculates value of waiting using payoffs from next period
    PP = (pu*AP[1:(n-i+1)] + pd*AP[0:(n-i)])/a
    # AP is the max of ealry exercise and waiting
    AP = np.maximum(P,PP)
    # ex is prices where early exercise is optimal
    ex = S[AP-PP>0]
    # eb calculates the highest price
    # where exercise is optimal to plot boundary
    if ex.shape[0]>0:
        eb[n-i] = ex.max()
    else:
        eb[n-i] = np.nan

print('The value of the American Put is=',AP[0])        
     # plot the exercise boundary
plt.figure(figsize=(9,6))
plt.plot(dt*np.arange(n+1),eb)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time
from math import pow, exp, sqrt
# parameters
# number of steps
n = 100
# interest rate
r = .1
# volatility
sig = .2
# Initial Stock Price
S0 = 42
# Strike Price
K = 42
# Maturity
T = 0.5

# calculate parameters for trinomial model
dt = T/n
a = np.exp(r*dt)
u = np.exp(sig*np.sqrt(3*dt))
d = 1/u
pu = 1/6 + np.sqrt(dt/(12*sig**2))*(r - sig**2/2)
pm = 2/3
pd = 1 - pu - pm
# Build vector of ending values
# and prices for which put is exercised
vec = np.arange(2*n+1)
vec1 = np.ones(2*n+1)
S = S0 * u**(vec-n*vec1)
AP = np.maximum(K-S,0)
ex = S[AP>0]
# eb is an array to save the boundary price
eb = np.zeros(n+1)
eb[n] = ex.max()
# Backward recursion in the loop
for i in range(n):
    vec = np.arange(2*(n-i-1)+1)
    vec1 = np.ones(2*(n-i-1)+1)
    # Possible Stock prices at times-i period
    S = S0 * u**(vec-(n-i-1)*vec1)
    # P calculates the value of early exercise
    P = np.maximum(K - S, 0)
    # PP calculates value of waiting using payoffs from next period
    PP = (pu*AP[2:(2*(n-i)+1)] + pm*AP[1:(2*(n-i))] + pd*AP[0:(2*(n-i)-1)])/a
    # AP is the max of ealry exercise and waiting
    AP = np.maximum(P,PP)
    # ex is prices where early exercise is optimal
    ex = S[(AP-PP)>0]
    # eb calculates the highest price
    # where exercise is optimal to plot boundary
    if ex.shape[0]>0:
        eb[n-i] = ex.max()
    else:
        eb[n-i] = np.nan
print('The American put price is=', AP[0])
# plot the exercise boundary
plt.figure(figsize=(10,7))
plt.scatter(dt*np.arange(n+1),eb)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time
from math import pow, exp, sqrt
# parameters
# number of steps
n = 100
# interest rate
r = .1
# volatility
sig = .2
# Initial Stock Price
S0 = 42
# Strike Price
K = 42
# Maturity
T = 0.5

# calculate parameters for trinomial model
dt = T/n
a = np.exp(r*dt)
u = np.exp(sig*np.sqrt(3*dt))
d = 1/u
pu = 1/6 + np.sqrt(dt/(12*sig**2))*(r - sig**2/2)
pm = 2/3
pd = 1 - pu - pm
# Build vector of ending values
# and prices for which put is exercised
ex = np.zeros(n+1)
S = np.zeros(2*n+1)
AP = np.zeros(2*n+1)

for j in range(2*n+1):
    S[j] = S0*u**(j-n)
    AP[j] = max(K-S[j],0)
    if AP[j]>0:
        ex[n] = S[j]

In [ ]:
for i in range(n):
    S = np.zeros(2*(n-i-1)+1)
    P = np.zeros(2*(n-i-1)+1)
    PP = np.zeros(2*(n-i-1)+1)
    for j in range(2*(n-i-1)+1):
        S[j] = S0*u**(j-(n-i-1))
        #
        # P calculates the value of early exercise
        P[j] = max(K-S[j],0)
        #
        # PP calculates value of waiting using payoffs
        # from next period
        PP[j] = (pu*AP[j+2] + pm*AP[j+1] + pd*AP[j])/a
        #
        # AP is the max of ealry exercise and waiting
        AP[j] = max(P[j],PP[j])
        #
        # ex is price where early exercise is optimal
        if P[j] > PP[j]:
            ex[n-i] = S[j]

print('The American put price is =', AP[0])            
# plot the exercise boundary
plt.figure(figsize=(10,7))
plt.scatter(dt*np.arange(n+1),ex)            


In [ ]:
# uses blackscholes(S0, K, r, q, sig, T, call = False)

def binomialbd(n,r,sig,S0,y,K,T):

    # parameters
    # number of steps
    #n 
    # interest rate
    #r 
    # volatility
    #sig 
    # Initial Stock Price
    #S0
    # Strike Price
        #K 
    # Maturity
    #T 
    # dividend yield
    # y

    # calculate parameters for binomial model
    dt = T/n
    delt = np.exp(-y*dt)
    a = np.exp(r*dt) * delt
    u = np.exp(sig*np.sqrt(dt))
    d = 1/u
    pu = (a-d)/(u-d)
    pd = 1-pu
    # Build vector of ending values
    # 
    
    S = np.zeros(n)
    AP = np.zeros(n)
    
    # Build vector of ending values
    # at the next to last date (penultimate date)
    vec = np.arange(n)
    vec1 = np.ones(n)

    S = S0 * u**(2*vec - (n-1)*vec1)

    AP = np.maximum(K-S,blackscholes(S, K, r, q, sig, dt, call = False))
    
    
    # Backward recursion in the loop
    for i in range(n-1):
        vec = np.arange(n-i-1)
        vec1 = np.ones(n-i-1)
        # Possible Stock prices at times-i period
        S = S0 * u**(2*vec-(n-i-1)*vec1+1)
    
    #     S = S0 * u**(2*vec-(n-i))
        # P calculates the value of early exercise
        P = np.maximum(K*vec1 - S, 0)
        # PP calculates value of waiting using payoffs from next period
        PP = (pu*AP[1:(n-i)] + pd*AP[0:(n-i-1)])/a
        # AP is the max of ealry exercise and waiting
        AP = np.maximum(P,PP)
        

    
    
    
    return AP[0]          

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time
from math import pow, exp, sqrt


#inputs
#number of steps= n; must be an even number
n=20
# interest rate
r=.1
# volatility
sig=.2
# initial stock price
S0=42
# dividend yield
y=0
# Strike price 
K=42
# expiration
T=0.5

if 2*int(n/2) == n:

    y2=binomialbd(n,r,sig,S0,y,K,T)
    y1=binomialbd(int(n/2),r,sig,S0,y,K,T)
    
    extrapolate= 2*y2-y1
    print('The extrapolated value=',extrapolate)

else:
    print('n must be even you big dummy!!')   